# Part 2: Data Cleaning, Lemmatization and Feature Extraction

### In this section, the data will be cleaned to remove undesirable information for NLP, on top of what was previously found in Part 1 (EDA).

### Then, text data will be combined and TF-IDF will be used to extract features.

In [ ]:
!{sys.executable} -m pip install pandas_profiling
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install nltk
!{sys.executable} -m spacy download en_core_web_md

In [47]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sb
import spacy
import nltk
import re

sb.set()

In [21]:
## edit this section accordingly
df = pd.read_csv(r"<>/sc1015-project/dataset/fake_job_postings.csv")
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [22]:
for column in df.columns:
    df[column]=df[column].fillna(f'missing_{column}')

We observe that there are 4 boolean valued columns (including `fraudulent`). However, they are stored as int64 and will need to be encoded into text for the nlp

In [23]:
boolean_cols = ['has_company_logo','has_questions','telecommuting']

# changing boolean values to text and appending their column names
for col in boolean_cols:
    df.loc[df[col]==0,col] = f'no_{col}'
    df.loc[df[col]==1,col] = f'yes_{col}'

df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,missing_salary_range,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,missing_benefits,no_telecommuting,yes_has_company_logo,no_has_questions,Other,Internship,missing_required_education,missing_industry,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,missing_salary_range,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,no_telecommuting,yes_has_company_logo,no_has_questions,Full-time,Not Applicable,missing_required_education,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",missing_department,missing_salary_range,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,missing_benefits,no_telecommuting,yes_has_company_logo,no_has_questions,missing_employment_type,missing_required_experience,missing_required_education,missing_industry,missing_function,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,missing_salary_range,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,no_telecommuting,yes_has_company_logo,no_has_questions,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",missing_department,missing_salary_range,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,no_telecommuting,yes_has_company_logo,yes_has_questions,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


Similarly, categorical values will be modified to the format of the boolean values (i.e {data value}_{col name})

In [24]:
categorical_cols = ['required_experience','required_education','employment_type']
for col in categorical_cols:
    val = list(df[col].values)
    val = ["_".join(re.findall(r"[\w']+",i)) + f"_{col}" for i in val]
    df[col] = np.array(val)
    
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,missing_salary_range,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,missing_benefits,no_telecommuting,yes_has_company_logo,no_has_questions,Other_employment_type,Internship_required_experience,missing_required_education_required_education,missing_industry,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,missing_salary_range,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,no_telecommuting,yes_has_company_logo,no_has_questions,Full_time_employment_type,Not_Applicable_required_experience,missing_required_education_required_education,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",missing_department,missing_salary_range,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,missing_benefits,no_telecommuting,yes_has_company_logo,no_has_questions,missing_employment_type_employment_type,missing_required_experience_required_experience,missing_required_education_required_education,missing_industry,missing_function,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,missing_salary_range,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,no_telecommuting,yes_has_company_logo,no_has_questions,Full_time_employment_type,Mid_Senior_level_required_experience,Bachelor's_Degree_required_education,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",missing_department,missing_salary_range,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,no_telecommuting,yes_has_company_logo,yes_has_questions,Full_time_employment_type,Mid_Senior_level_required_experience,Bachelor's_Degree_required_education,Hospital & Health Care,Health Care Provider,0


Now, the text of all the columns will be concatenated into a single block to be processed by tf-idf. The text will also be converted to lowercase to reduce variables that have little to no effect (in this case) on the meaning of the text.

In [44]:
list_of_cols = list(df)
list_of_cols.remove('fraudulent') # remove fraudulent because it is the desired classification
list_of_cols.remove('job_id') # remove job_id because useless data
print(list_of_cols)

ser=pd.Series(np.full(shape=(df.shape[0],), fill_value=''))
for col in list_of_cols:
    ser += (" " + df[col].str.lower())  # Convert text to lowercase

    
    
print(ser.describe())
ser.head()

['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
count                                                 17880
unique                                                17596
top        title insurance ops: sr title officer/counsel...
freq                                                      6
dtype: object


0     marketing intern us, ny, new york marketing m...
1     customer service - cloud video production nz,...
2     commissioning machinery assistant (cma) us, i...
3     account executive - washington dc us, dc, was...
4     bill review manager us, fl, fort worth missin...
dtype: object

We now have 17880 rows of text blocks, each representing a single job posting.


Here we will generate a list of stopwords, and exclude 'no' from it since this word encodes information regarding the boolean columns.

In [76]:
nltk.download('stopwords')
list_of_stopwords = nltk.corpus.stopwords.words('english')
list_of_stopwords.remove('no')

nlp = spacy.load('en_core_web_md')


def lemmatize_text(text):
    text = nlp(text)
    #-PRON- happens when the word is a pronoun. In that case, return the word again
    text = ' '.join([word.lemma_ if (word.lemma_ != '-PRON-' and word.lemma not in list_of_stopwords) else word.text for word in text])
    return text

ser_lemma = ser.apply(lambda x: lemmatize_text(x))

print(ser_lemma.describe())
ser_lemma.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Workstation\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


count                                                 17880
unique                                                17595
top         mirfield warehousing apprenticeship under na...
freq                                                      6
dtype: object


0       marketing intern us , ny , new york marketi...
1      customer service - cloud video production nz...
2      commission machinery assistant ( cma ) us , ...
3      account executive - washington dc us , dc , ...
4      bill review manager us , fl , fort worth mis...
dtype: object

In [78]:
ser_lemma.to_csv('lemmatized_text.csv')